In [45]:
import numpy as np
import pandas as pd

In [46]:
df = pd.read_csv("./data/coil_information.csv")
data_df = df.copy()
data_df.drop("Unnamed: 0", axis=1, inplace=True)
data_df = data_df.astype({"WDT_LEN" : "str", "WGT_WGT" : "str", "IND_CD" : "str", "OUD_LEN" : "str", "STL_CD" : "str"})

kind_of_ann = data_df["PNSPRC_CD"].unique().tolist()
kind_of_cycle = data_df["cycle"].unique().tolist()

print(kind_of_ann)
print(kind_of_cycle)
data_df.head()

['AN11', 'AN10', 'AN12', 'AN13']
['620', '750', '725', '700', '725(표준)', '작업지시서 소둔 Cycle 적용', '700(표준)', '735', '720', '700*Max', '725*Max', '750*Max']


,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사
0,09KA1B77311,SCP1-H-B,AN11,620,309.0,4811,nan,1668,10021,1282N,NaN
1,21M12D061111A,50CRV4M-S,AN10,750,248.0,2091,nan,1275,10022,1282S,유신정밀공업(주) / (주)화진금속
2,22A122851111A,S55CM-R,AN10,725,410.0,5121,nan,1511,10022,1282N,NaN
3,22A122852111A,S55CM-R,AN10,725,410.0,5121,nan,1511,10022,1282N,NaN
4,22A122853111A,S55CM-R,AN10,725,210.0,2623,nan,1511,10022,1282N,NaN


In [47]:
# for i in range(len(kind_of_ann)):
#     for j in range(len(kind_of_cycle)): 
#         separated_df = data_df.loc[(data_df['PNSPRC_CD'] == kind_of_ann[i]) & (data_df['cycle'] == kind_of_cycle[j])]
#         if separated_df.shape[0] >= 1 : print(separated_df.shape[0])
#         # ann차수, cycle을 통한 여러개의 조합들 중, 해당하는 열이 없는 조합일 때는 그냥 스킵만 해줄까 ?


# 첫번째 코일 그룹
separated_df = data_df.loc[(data_df['PNSPRC_CD'] == 'AN11') & (data_df['cycle'] == '620')]
middle_df = separated_df.loc[separated_df['IND_CD'] == '508']
first_coil_group = middle_df
print(first_coil_group.shape)
first_coil_group.tail()

(0, 11)


,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사


In [48]:
coil_number = first_coil_group["COIL_NO"].tolist()
ann_number = first_coil_group["PNSPRC_CD"].tolist()
cycle = first_coil_group["cycle"].tolist()
coil_heights = first_coil_group["WDT_LEN"].tolist()
coil_weights = first_coil_group["WGT_WGT"].tolist()
coil_inner = first_coil_group["IND_CD"].tolist()
coil_outer = first_coil_group["OUD_LEN"].tolist()
coil_emergency = first_coil_group["EMG_CD"].tolist()

In [49]:
base_capacity_information = pd.read_csv("./data/base_capacity_information.csv")

In [50]:
first_base_group = base_capacity_information.loc[(base_capacity_information['Maker'] == 'EBNER') & (base_capacity_information['Base_number'] <= 6)]
first_base_group

,Maker,Base_number,Weight(Ton),Height(mm),Outer_max(mm),Outer_min(mm),Inner(mm)
0,EBNER,1,35,2800,1700,900,508
1,EBNER,2,35,2800,1700,900,508
2,EBNER,3,35,2800,1700,900,508
3,EBNER,4,35,2800,1700,900,508
4,EBNER,5,35,2800,1700,900,508
5,EBNER,6,35,2800,1700,900,508


In [51]:
# 첫번째 베이스그룹
base_weight_capa = first_base_group["Weight(Ton)"].tolist()
base_height_capa = first_base_group["Height(mm)"].tolist()
base_min_od_capa = first_base_group["Outer_max(mm)"].tolist()
base_max_od_capa = first_base_group["Outer_min(mm)"].tolist()
base_id_capa = first_base_group["Inner(mm)"].tolist()

In [52]:
import pandas as pd
import numpy as np
import ortools
from ortools.linear_solver import pywraplp


solver = solver = pywraplp.Solver.CreateSolver('SCIP')

# data_df = pd.read_csv("./data/data_df.csv")
# data_df.drop("Unnamed: 0", axis=1, inplace=True)
# coil_df = data_df.copy()
#data_df = data_df.astype({"WDT_LEN" : "str", "WGT_WGT" : "str", "IND_CD" : "str", "OUD_LEN" : "str", "STL_CD" : "str"})



# Coil
data = {}
# coil_number = first_coil_group["COIL_NO"].tolist()
# ann_number = first_coil_group["PNSPRC_CD"].tolist()
# cycle = first_coil_group["cycle"].tolist()
# coil_heights = first_coil_group["WDT_LEN"].tolist()
# coil_weights = first_coil_group["WGT_WGT"].tolist()
# coil_inner = first_coil_group["IND_CD"].tolist()
# coil_outer = first_coil_group["OUD_LEN"].tolist()
# coil_emergency = first_coil_group["EMG_CD"].tolist()


assert len(coil_number) == len(ann_number) == len(cycle) == len(coil_heights) == len(coil_weights) == len(coil_inner) == len(coil_outer) == len(coil_emergency) 
data['coil_number'] = coil_number
data['ann_number'] = ann_number
data['cycle'] = cycle
data['coil_heights'] = coil_heights
data['coil_weights'] = coil_weights
data['coil_inner'] = coil_inner
data['coil_outer'] = coil_outer
data['coil_emergency'] = coil_emergency
data['coils'] = list(range(len(coil_weights)))
data['num_coils'] = len(coil_number)

# Base
base_info = pd.read_csv("./data/base_capacity_information.csv")
base_df = base_info.copy()



# base_maker = base_df['Maker'].tolist()
# base_number = base_df['Base_number'].tolist()
base_weights = first_base_group['Weight(Ton)'].tolist()
base_heights = first_base_group['Height(mm)'].tolist()
base_outer_max = first_base_group['Outer_max(mm)'].tolist()
base_outer_min = first_base_group['Outer_min(mm)'].tolist()
base_inner = first_base_group['Inner(mm)'].tolist()


# data['base_maker'] = base_maker
# data['base_number'] = base_number
data['base_weights'] = base_weights
data['base_heights'] = base_heights
data['base_outer_max'] = base_outer_max
data['base_outer_min'] = base_outer_min
data['base_inner'] = base_inner

number_base = len(base_weights)

#I wanted to change the values at a later data
data['bases'] = list(range(number_base))

assert len(data['base_weight']) == number_base
assert len(data['base_weight']) == len(data['base_height']) == len(data['base_outer_max']) == len(data['base_outer_min']) == len(data['base_inner'])

print("coil_number: ",*data['coil_number'][:5])
print('coil_heights:',*data['coil_heights'][:5])
print('coil_weights:',*data['coil_weights'][:5])
print('coil_inner:', *data['coil_inner'][:5])
print('coil_outer:', *data['coil_outer'][:5])
print("Number of coils:", len(coil_number+1))
print("Number of Bases:" , number_base)
# print('Knapsack Capacities: 50 Pounds, 50 cubic inches, 5 Levels of Radiation')


x = {}
for i in data['coils']:
    for j in data['bases']:
        x[(i,j)] = solver.IntVar(0,1,'x_%i_%i' % (i, j))


#Constraints
for i in data['coils']:
    solver.Add(sum(x[i,j] for j in data['bases'])<=1)


for j in data['bases']:
    solver.Add(sum(x[(i,j)]*data['coil_weights'][i] 
                  for i in data['coils']) <= data['base_weights'][j])


for j in data['bases']:
    solver.Add(sum(x[(i,j)]*data['coil_heights'][i] 
                  for i in data['coils']) <= data['base_heights'][j])

# TODO : 외경조건은 sum이 아니므로 그 문법에 맞게 코드 수정
for j in data['bases']:
    solver.Add((x[(i,j)]*data['coil_outer'][i] for i in data['coils']) <= data['base_outer_max'][j])


for j in data['bases']:
    solver.Add((x[(i,j)]*data['coil_outer'][i] for i in data['coils']) >= data['base_outer_min'][j])



# objective function
objective = solver.Objective()
for i in data['coils']:
    for j in data['bases']:
        objective.SetCoefficient(x[(i,j)], data['coil_heights'][i])
objective.SetMaximization()


# Solve
solv = solver.Solve()
if solv == pywraplp.Solver.OPTIMAL:
    print('Total Batched Heights:', objective.Value())
    total_weight = 0
    for j in data['bags']:
        base_weights = 0
        base_heights = 0
        base_outer_max= 0
        base_outer_min = 0
        print('\n','Base', j+1 , '\n')
        for i in data['items']:
            if x[i,j].solution_value()>0:
                print('Item:', i , 
                      'coil_heights',data['coil_heights'][i],
                      'coil_weights', data['coil_weights'][i],
                    #   'coil_inner', data['coil_inner'][i],
                      'coil_outer', data['coil_outer'][i],
                    #   'coil_emergency',data['coil_emergency'][i]
                     )
                base_weights += data['base_weights'][i]
                base_heights += data['base_heights'][i]
        print('Packed Knapsack Value: ', base_weights)
        print('Packed Knapsack Weight: ', base_heights)
else:
    print("There is no optimal solution")

ModuleNotFoundError: No module named 'ortools'